<a href="https://colab.research.google.com/github/Aanantya/nlp/blob/main/generating_poetic_texts_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
# downloading 'shakespeare.txt', text corpus data to work with

filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# read, convert the text content to lower-case
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
# training the model on a specific portion of text data, eg. from index:300000 -> index:800000
text = text[300000:800000]

In [6]:
# listing and sorting the text data as a sequence of characters
characters = sorted(set(text))

In [7]:
# creating the dictionary to map every 'char-to-index' and 'index-to-char'
char_to_index = dict((c, i) for i, c in enumerate(characters))

index_to_char = dict((i, c) for i, c in enumerate(characters))

In [8]:
# the approach to learn from the data...
# text will be split into sentence len of (start_index->to->40 char), the start_index = start_index + STEP_SIZE
# next_character = start_index + SEQUENCE_LEN, i.e the next_character will point towards the next char of the sentence sequence
# eg. sentences[0] = 'first citizen:\nbefore we proceed any fur'
#     next_characters[0] = 't'

SEQUENCE_LEN = 40
STEP_SIZE = 3

sentences = []
next_characters = []

for i in range(0, len(text) - SEQUENCE_LEN, STEP_SIZE):
  sentences.append(text[i: i+SEQUENCE_LEN])
  next_characters.append(text[i+SEQUENCE_LEN])

In [10]:
# initializing a zeros matrix for fixed size sentence as input and expected next_character as output
x = np.zeros((len(sentences), SEQUENCE_LEN, len(characters)))
y = np.zeros((len(sentences), len(characters)))

print(x.shape, y.shape)

(166654, 40, 39) (166654, 39)


In [11]:
# defining the training data with 1/0
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_to_index[char]] = 1

  y[i, char_to_index[next_characters[i]]] = 1

In [12]:
# Training the model
# LSTM for memory
# Dense layer for the hidden layer
# Activation layer for output layer
# optimizer to compile all the layers

model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LEN, len(characters)))) # first layer
model.add(Dense(len(characters))) # hidden layers with max neuron size = len(charcatres) -> 39
model.add(Activation('softmax')) # output layer

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=10)

Epoch 1/10
651/651 [==============================] - 103s 155ms/step - loss: 2.1533
Epoch 2/10
651/651 [==============================] - 98s 151ms/step - loss: 1.7297
Epoch 3/10
651/651 [==============================] - 98s 151ms/step - loss: 1.5993
Epoch 4/10
651/651 [==============================] - 100s 154ms/step - loss: 1.5285
Epoch 5/10
651/651 [==============================] - 98s 151ms/step - loss: 1.4806
Epoch 6/10
651/651 [==============================] - 98s 150ms/step - loss: 1.4451
Epoch 7/10
651/651 [==============================] - 101s 155ms/step - loss: 1.4174
Epoch 8/10
651/651 [==============================] - 98s 151ms/step - loss: 1.3960
Epoch 9/10
651/651 [==============================] - 99s 152ms/step - loss: 1.3781
Epoch 10/10
651/651 [==============================] - 99s 153ms/step - loss: 1.3602


In [13]:
# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               86016     
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
 activation (Activation)     (None, 39)                0         
                                                                 
Total params: 91047 (355.65 KB)
Trainable params: 91047 (355.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [17]:
# save model as 'textgenerator.model'
model.save("textgenerator.model")

In [21]:
# loading the saved 'textgenerator.model'
model = tf.keras.models.load_model('textgenerator.model')


# helper function offered by keras which help choice between the choices offered by softmax function
# depending upon 'temperature' the value can be conservetive or experimental
# temperature = 0 -> safe pick; temperature = 1 -> experimental choice(possible that selection won't make sense)
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  prob = np.random.multinomial(1, preds, 1)
  return np.argmax(prob)


def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQUENCE_LEN - 1) # leaving the text length for atleast one sentence processing
  generated_text = "" # empty
  sentence = text[start_index: start_index + SEQUENCE_LEN]  # pick sentence starting from random index
  generated_text += sentence  # base senetence to work on

  for i in range(length):
    x = np.zeros((1, SEQUENCE_LEN, len(characters)))
    for t, char in enumerate(sentence):
      x[0, t, char_to_index[char]] = 1  # sentence for prediction


    prediction = model.predict(x, verbose=0)[0]

    next_index = sample(prediction, temperature) # pick from prediction

    next_character = index_to_char[next_index]  # convert index-to-char value

    generated_text += next_character  # add char to base sentence

    sentence = sentence[1:] + next_character # shift the sentence window

  return generated_text

In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               86016     
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
 activation (Activation)     (None, 39)                0         
                                                                 
Total params: 91047 (355.65 KB)
Trainable params: 91047 (355.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
# outputs at different temperature values
print('-------------(300, 0.2)-------------')
print(generate_text(300, 0.2))
print('-------------(300, 0.4)-------------')
print(generate_text(300, 0.4))
print('-------------(300, 0.6)-------------')
print(generate_text(300, 0.6))
print('-------------(300, 0.8)-------------')
print(generate_text(300, 0.8))
print('-------------(300, 1.0)-------------')
print(generate_text(300, 1.0))

-------------(300, 0.2)-------------
over as they were gods or goddesses; you have the day the lands
and the still the store of the world that they
and the rest with the world that the stars
the father hath a prince the looks of his life,
and then the hands the tides the store of the still.

polixenes:
now the comes the starment to the beart
the part the starst to see the gr
-------------(300, 0.4)-------------
ear, delivered with a groan,
'o, farewell the still for the marrester'd the still.

romeo:
then will not still be setten a something lands
and the lady speak the run suddest me a ster

duke of york:
sir, to see thy same the news with a summore.

northumberland:
what serve the hands me as the warth of the still
the compliet, and to part th
-------------(300, 0.6)-------------
oncord of my state and time
had not an earion and the 'ating sterity.

king richard ii:
be honouriet, and the bay thou wert with our mards,
shall tears, my lord, the person as here,
i have as my gracious fro

Reference: https://www.neuralnine.com/generating-texts-with-recurrent-neural-networks-in-python/